In [1]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np
import geopandas as gpd
import fiona

### Plan

* start from all shapes for a single operator
* for each shape, assess for each hour of day x weekday/sat/sun (with day of week/time of day cols):
    * (can loop over weekday/sat/sun here at trip+stop_times join)
    * existing frequency at midpoint stop
    * current total runtime
* preserve route_id in main table
* (seperately) calculate operator/routes/shapes in each Census tract
* can then join tracts to service info; characterize route service target geographically
    * generate additional frequencies/service hours/service miles for service target
* (optional) generate hypothetical trips table
* (optional) assign service hours/miles to tracts (not sure why we'd need this yet)

In [51]:
def find_midpoint_stop(shape, trips, stop_times):
    return

In [101]:
weekday = (tbl.views.gtfs_schedule_fact_daily_trips()
     >> select(_.calitp_itp_id, _.service_date, _.trip_key, _.trip_id, _.is_in_service)
     >> filter(_.calitp_itp_id == 300)
     >> filter(_.service_date == '2021-10-07')
     >> filter(_.is_in_service == True)
     # >> show_query()
     >> collect())

In [102]:
saturday = (tbl.views.gtfs_schedule_fact_daily_trips()
     >> select(_.calitp_itp_id, _.service_date, _.trip_key, _.trip_id, _.is_in_service)
     >> filter(_.calitp_itp_id == 300)
     >> filter(_.service_date == '2021-10-09')
     >> filter(_.is_in_service == True)
     # >> show_query()
     >> collect())

In [47]:
sunday = (tbl.views.gtfs_schedule_fact_daily_trips()
     >> select(_.calitp_itp_id, _.service_date, _.trip_key, _.trip_id, _.is_in_service)
     >> filter(_.calitp_itp_id == 300)
     >> filter(_.service_date == '2021-10-10')
     >> filter(_.is_in_service == True)
     # >> show_query()
     >> collect())

In [99]:
weekday

,calitp_itp_id,service_date,trip_key,trip_id,is_in_service
0,300,2021-10-07,6925765546125290544,860314,True
1,300,2021-10-07,-1928001016074065841,860319,True
2,300,2021-10-07,7833843909345812156,860392,True
3,300,2021-10-07,-5177588935140610120,860211,False
4,300,2021-10-07,1402947890318209155,857609,False
...,...,...,...,...,...
3220,300,2021-10-07,-972320320158977901,857023,False
3221,300,2021-10-07,-3988614873976181013,857147,True
3222,300,2021-10-07,5539471939284617726,857193,True
3223,300,2021-10-07,5732727140221153573,857188,True


In [100]:
saturday

,calitp_itp_id,service_date,trip_key,trip_id,is_in_service
0,300,2021-10-09,-2507121095560876263,860372,False
1,300,2021-10-09,6866201105861985504,860369,False
2,300,2021-10-09,-7621960199486777685,860384,False
3,300,2021-10-09,7320376209098066520,860332,False
4,300,2021-10-09,-458251259243686914,857674,False
...,...,...,...,...,...
3220,300,2021-10-09,8066106239632692258,856940,True
3221,300,2021-10-09,2615191695776342177,857013,False
3222,300,2021-10-09,-8951696881711047180,857041,False
3223,300,2021-10-09,-2388154850293022578,856918,True


In [50]:
sunday

,calitp_itp_id,service_date,trip_key,trip_id,is_in_service
0,300,2021-10-10,-1566067371968892885,857665,True
1,300,2021-10-10,-2101857464276218548,857728,True
2,300,2021-10-10,-418902018110506047,857737,True
3,300,2021-10-10,-1625451368464181312,856247,True
4,300,2021-10-10,-9084693455007451867,856268,True
...,...,...,...,...,...
765,300,2021-10-10,-8389758290909726702,856982,True
766,300,2021-10-10,-5669199107012448594,856955,True
767,300,2021-10-10,-4817212099725408206,857050,True
768,300,2021-10-10,-8679058570687179377,857063,True


In [38]:
sunday.trip_id.unique()

array(['860329', '860390', '860348', ..., '857161', '857152', '857168'],
      dtype=object)

In [44]:
weekday['trip_id'].isin(sunday.trip_id).value_counts()

True    1484
Name: trip_id, dtype: int64

In [42]:
weekday.trip_id.is_unique

True

In [36]:
tbl.views.gtfs_schedule_dim_trips() >> filter(_.trip_key.isin(df.trip_key))

,calitp_itp_id,calitp_url_number,route_id,service_id,trip_id,shape_id,trip_headsign,trip_short_name,direction_id,block_id,wheelchair_accessible,bikes_allowed,calitp_extracted_at,calitp_hash,trip_key,calitp_deleted_at
0,300,0,3329,10,856323,25320,UCLA,None,0,None,0,0,2021-09-01,SUNt6O6sEOlmqF7TDcy9aQ==,-6931215805910091774,2021-10-11
1,300,0,3338,10,859874,25352,Palms Rapid,None,1,None,0,0,2021-09-01,DfXpl4agd/nBkq4ssu3jvA==,5548948652349145677,2021-10-11
2,300,0,3328,10,856037,25314,UCLA,None,0,None,0,0,2021-09-01,QLrFr1A0KanMD9CfSOEgzg==,3991304789260610440,2021-10-11
3,300,0,3329,10,856342,25321,UCLA,None,0,None,0,0,2021-09-01,ugcELfTTZBLD6GZw8HX2Pg==,4194119154593412596,2021-10-11
4,300,0,3339,1,857922,25354,Brentwood,None,0,None,0,0,2021-09-01,vrGGszkkXyrjYHUy6kXHoA==,3893897006637506314,2021-10-11


In [78]:
weekday_st = (tbl.views.gtfs_schedule_data_feed_trip_stops()
              >> select(_.route_id, _.stop_time_key, _.stop_id, _.trip_key)
              >> filter(_.trip_key.isin(weekday.trip_key))
             )
              # >> join(_, tbl.views.gtfs_schedule_dim_stop_times(), on='stop_time_key')
              # >> collect())

In [74]:
weekday_st

,route_id,stop_time_key,stop_id,trip_key
0,3337,-327170457733062793,1028,8573725933419801218
1,3337,1973147268010187735,227,8573725933419801218
2,3337,8468264295819346454,941,-2051161304346219309
3,3337,-1901651974668897562,962,8573725933419801218
4,3337,-7842200365246461971,966,8730074154597276398
...,...,...,...,...
49923,3352,1141080671212585684,1344,6866201105861985504
49924,3352,7860941725990749510,1315,-1160266126176692376
49925,3352,-2921874392368385058,1344,8809784104098515172
49926,3352,-4081401008365419912,1310,3880834598261698584


In [77]:
weekday_st.stop_time_key.is_unique

True

In [81]:
df = (tbl.views.gtfs_schedule_dim_stop_times()
      >> select(_.trip_id, _.stop_sequence, _.departure_time, _.stop_time_key)
      # >> filter(_.stop_time_key.isin(weekday_st.stop_time_key))
      >> join(_, weekday_st, on = 'stop_time_key')
      >> collect())

In [82]:
df

,trip_id,stop_sequence,departure_time,stop_time_key,route_id,trip_key,stop_id
0,857575,1,06:00:00,-5876918209241522447,3337,-7392505570042088157,1512
1,857575,7,06:04:33,8550656125231073096,3337,-7392505570042088157,1234
2,857578,13,15:56:06,-6138407760280451670,3337,8730074154597276398,980
3,857578,15,15:58:00,7224737784157703076,3337,8730074154597276398,111
4,857580,19,19:06:05,-5022537002904565299,3337,7280416254057548872,983
...,...,...,...,...,...,...,...
49923,860382,7,16:27:00,8393948976742153144,3352,4953430223475235812,1344
49924,860322,7,10:19:00,5477497119918317737,3352,-8543392098392464785,1301
49925,860314,7,07:36:00,192615579256497030,3352,6925765546125290544,1301
49926,860323,7,10:39:00,4412881298630523507,3352,-4583309100422833604,1301


## keep middle for each

In [95]:
middle_stops = df >> group_by(_.trip_key) >> summarize(middle_stop = _.stop_sequence.median())
middle_stops.middle_stop = middle_stops.middle_stop.astype('int64')

In [96]:
middle_stops

,trip_key,middle_stop
0,-9215696245559518014,26
1,-9197945165465699095,9
2,-9188837691433697423,9
3,-9175735659033559743,18
4,-9110424570004612463,16
...,...,...
1479,9106770440781177926,4
1480,9176251286402730697,28
1481,9199748749186205072,12
1482,9210101050081509882,16
